# 知识解析
主要是添加一些同义词和反义词的特征

In [0]:
class Antonyms:
  def __init__(self, filepath = './Antonyms.txt'):
    with open(filepath, encoding='utf-8') as antonyms_file:
      self.antonyms_dict = [line.split()[1:] for line in antonyms_file.readlines()]
  
  def judge_antonyms(self, word1, word2):
    # 同一个词不关注
    if word1 == word2:
      return 0
    for anton_set in self.antonyms_dict:
      if word1 in anton_set and word2 in anton_set:
        return 1
    # 找不到被标记为0，在之后的计算中就忽略
    return 0

In [25]:
antonyms = Antonyms()
antonyms.judge_antonyms('正义','邪恶')

0

In [0]:
class Synonyms:
  def __init__(self, filepath = './Synonyms.txt'):
    with open(filepath, encoding='gbk') as synonyms_file:
      self.synonyms_dict = [line.split()[1:] for line in synonyms_file.readlines()]
  def judge_synonyms(self, word1, word2):
    if word1 == word2:
      return 0
    for synon_set in self.synonyms_dict:
      if word1 in synon_set and word2 in synon_set:
        return 1
    return 0

In [27]:
synonyms = Synonyms()
synonyms.judge_synonyms('助理','助手')

1

# 特征提取

In [0]:
import jieba

In [0]:
# bigram语言模型
class Bigram():
    def __init__(self, sentence_list):
      word_dict = {} # 统计词频
      bigram_dict = {} # 统计两个词连接的频率
      # 把sentece切分成2个词一组
      for sentence in sentence_list:
        word_list = list(jieba.cut(sentence))
        for idx in range(len(word_list)-1): # 对句子的每个词进行枚举。此处不统计最后一个词，因为最后一个词无后继词
          word, next_word = word_list[idx], word_list[idx+1]
          # print(idx,word)
          if word in word_dict: # 对一个词统计词频
            word_dict[word] += 1
          else:
            word_dict[word] = 1
          if word in bigram_dict: # 对一对词统计词频
            if next_word in bigram_dict[word]:
              bigram_dict[word][next_word] += 1 # 当前词和后继词都在词典中
            else:
              bigram_dict[word][next_word] = 1 # 当前词在，后继词不在
          else:
            bigram_dict[word] = {next_word:1} # 当前词和后继词都不在
      # 初始化单词频数和双词频数
      self.word_dict = word_dict # 初始化单词频数和双词频数
      self.bigram_dict = bigram_dict
      self.word_amount = 0 # 统计总词数
      for word in self.word_dict:
        self.word_amount += self.word_dict[word]

    # 获得分词结果所得到的概率
    def _calc_possibility(self, segmentation):
      words_list = segmentation.split(' ')
      p = 1.0
      length = len(words_list)
      left_adjacent_word = None
      for word in words_list:
        if left_adjacent_word == None: # 第一个词，没有左邻词，直接计算概率
          if word in self.word_dict:
            # 词在词典中，p=本词出现数+1/总词数+词表大小
            p *= (self.word_dict[word] + 1.0) / (self.word_amount + len(self.word_dict))
          else:
            # 词不在词典中，p=1/总词数+词表大小
            p *= 1.0 / (self.word_amount + len(self.word_dict))
        else: # 有左邻词，计算条件概率
          if left_adjacent_word in self.bigram_dict:
            if word in self.bigram_dict[left_adjacent_word]:
              # 左邻词和本词都在词典中， p=左邻词和本词共现数+1/左邻词出现数+词表大小
              p *= (self.bigram_dict[left_adjacent_word][word] + 1.0) / (self.word_dict[left_adjacent_word] + len(self.word_dict))
            else:
              # 本词不在词典中，左邻词和本词共现数为0， p=1/左邻词出现数+词表大小
              p *= 1.0 / (self.word_dict[left_adjacent_word] + len(self.word_dict))
          else:
            # 左邻词不在词典中，左邻词出现数和本词共现数均为0。如果按拉普拉斯平滑，共现数为1，出现数为词表大小，p=1/词表大小。但这个概率比左邻词出现本词不出现的概率还大。
            # 我们假设语料已经体现了现实的分布，则左邻词不出现（OOV）意味着一个非常小概率的情况。在这里我们设置为p=1/(词表大小*词表大小)
            p *= 1.0 / (len(self.word_dict) * len(self.word_dict))
      return p

In [30]:
bigram = Bigram(sentences_corpus)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.913 seconds.
Prefix dict has been built successfully.


In [0]:
# 特征1：根据bigram计算两文本重复和不同的得分
def index_bigram_overlap(t1, t2, bigram):
  def _create_bi(word_list):
    word_list.append('</s>')
    text_bi = []
    last_word = '<s>'
    for word in word_list:
      text_bi.append((last_word, word))
      last_word = word
    return text_bi
  t1_bi = _create_bi(t1)
  t2_bi = _create_bi(t2)
  prop_comm = 0.
  prop_t2 = 0.
  for word_bi in t2_bi:
    prop = bigram._calc_possibility('{0} {1}'.format(word_bi[0],word_bi[1]))
    prop_t2 += prop
  if word_bi in t1_bi:
    prop_comm += prop
  return prop_comm / prop_t2

In [0]:
# 特征2：余弦相似度
def index_cosine_score(t1, t2):
  words = []
  for word in t1:
    if word not in words:
      words.append(word)
  for word in t2:
    if word not in words:
      words.append(word)
  # calculate vector
  vec1 = np.zeros(len(words))
  vec2 = np.zeros(len(words))
  for word in t1:
    idx = words.index(word)
    vec1[idx] += 1
  for word in t2:
    idx = words.index(word)
    vec2[idx] += 1
  return np.dot(vec1,vec2)/(np.linalg.norm(vec1)*(np.linalg.norm(vec2)))

In [0]:
# 特征3：最长交叠子串
def index_longest_overlap_substring(t1, t2):
    def los_len(t1, t2):
        len1 = len(t1)
        len2 = len(t2)
        c = np.zeros((len1+1, len2+1))
        for i in range(len1):
            for j in range(len2):
                if t1[i] == t2[j]:
                    c[i + 1][j + 1] = c[i][j] + 1
                elif c[i + 1][j] > c[i][j + 1]:
                    c[i + 1][j + 1] = c[i + 1][j]
                else:
                    c[i + 1][j + 1] = c[i][j + 1]
        return c[len1][len2]
    return los_len(t1, t2) * 1. / len(t2)

In [0]:
# 特征4：反义词指数
def index_antonym(t1, t2, antonyms):
  num = 0
  for word2 in t2:
    for word1 in t1:
      if antonyms.judge_antonyms(word2, word1) == 1:
        num += 1
  return num / len(word2)

# 特征5：同义词指数
def index_synonym(t1, t2, synonyms):
  num = 0
  for word2 in t2:
    for word1 in t1:
      if synonyms.judge_synonyms(word2, word1) == 1:
        num += 1
        break
  return num / len(word2)

In [0]:
# 特征6：连接词"和"
def index_and(t1, t2):
  word_and = ['和', '与', '并', '且', '及']
  flag1 = False
  flag2 = False
  for word in word_and:
    if word in t1:
      flag1 = True
  for word in word_and:
    if word in t2:
      flag2 = True
  if not flag1 and flag2:
    return 1
  return 0

# 特征7：连接词"或"
def index_or(t1, t2):
  word_or = ['或']
  flag1 = False
  flag2 = False
  for word in word_or:
    if word in t1:
      flag1 = True
  for word in word_or:
    if word in t2:
      flag2 = True
  if not flag1 and flag2:
    return 1
  return 0

In [0]:
# 加了特征8和9以后正确率反而下降了......
# 特征8：t2是否有否定词
def index_neg(t1, t2):
  word_neg = ['不是', '不可', '没有', '不要', '不能', '不得', '不让', '不应该' ,'否定', '并非', '禁止']
  flag_t1 = False
  flag_t2 = False
  for word in t2:
    if word in word_neg:
       flag_t2 = True
  for word in t1:
    if word in word_neg:
       flag_t1 = True
  if flag_t1 is not flag_t2:
    return 1
  return 0

In [0]:
# 特征9：是否有'可能'变'一定'或'一定'变'可能'
def index_possible(t1, t2):
  list_of_may = ['可能', '好像', '或者', '大约', '粗略', '概略', '大体', '大致', '不定', '大致', '大抵', '或许', '约略', '简略', '八成', '大要', '也许', '大略', '不一定']
  list_of_must = ['一定', '肯定', '必须', '必需', '必然', '确定', '注定', '必定', '详细', '确乎', '周密', '细致', '具体', '不会']
  def _judge(text, word):
     for tmp_word in text:
        if tmp_word in word:
           return True
     return False
  flag_t1_may = _judge(t1, list_of_may)
  flag_t1_must = _judge(t1, list_of_must)
  flag_t2_may = _judge(t2, list_of_may)
  flag_t2_must = _judge(t2, list_of_must)
  if (flag_t1_may is not flag_t2_may) or (flag_t1_must is not flag_t2_must):
     return 1
  return 0

# 构造训练集和验证集的特征向量

In [38]:
train_data = []
train_label = []
eval_data = []
eval_label = []
for i in range(len(pair_set['label'])):
  t1 = list(jieba.cut(pair_set['t1'][i]))
  t2 = list(jieba.cut(pair_set['t2'][i]))
  fea1 = index_bigram_overlap(t1, t2, bigram)
  fea2 = index_cosine_score(t1, t2)
  fea3 = index_longest_overlap_substring(t1, t2)
  fea4 = index_synonym(t1, t2, synonyms)
  fea5 = index_antonym(t1, t2, antonyms)
  fea6 = index_and(t1, t2)
  fea7 = index_or(t1, t2)
  fea8 = index_neg(t1, t2)
  fea9 = index_possible(t1, t2)
  x = np.random.randint(10) # 测试集80%，验证集20%
  if x < 8:
    train_data.append([fea1, fea2, fea3, fea4, fea5, fea6, fea7, fea8, fea9])
    train_label.append(pair_set['label'][i] == 'Y')
  else:
    eval_data.append([fea1, fea2, fea3, fea4, fea5, fea6, fea7, fea8, fea9])
    eval_label.append(pair_set['label'][i] == 'Y')
  print('{0} of {1} done.'.format(i, len(pair_set['label'])))

0 of 581 done.
1 of 581 done.
2 of 581 done.
3 of 581 done.
4 of 581 done.
5 of 581 done.
6 of 581 done.
7 of 581 done.
8 of 581 done.
9 of 581 done.
10 of 581 done.
11 of 581 done.
12 of 581 done.
13 of 581 done.
14 of 581 done.
15 of 581 done.
16 of 581 done.
17 of 581 done.
18 of 581 done.
19 of 581 done.
20 of 581 done.
21 of 581 done.
22 of 581 done.
23 of 581 done.
24 of 581 done.
25 of 581 done.
26 of 581 done.
27 of 581 done.
28 of 581 done.
29 of 581 done.
30 of 581 done.
31 of 581 done.
32 of 581 done.
33 of 581 done.
34 of 581 done.
35 of 581 done.
36 of 581 done.
37 of 581 done.
38 of 581 done.
39 of 581 done.
40 of 581 done.
41 of 581 done.
42 of 581 done.
43 of 581 done.
44 of 581 done.
45 of 581 done.
46 of 581 done.
47 of 581 done.
48 of 581 done.
49 of 581 done.
50 of 581 done.
51 of 581 done.
52 of 581 done.
53 of 581 done.
54 of 581 done.
55 of 581 done.
56 of 581 done.
57 of 581 done.
58 of 581 done.
59 of 581 done.
60 of 581 done.
61 of 581 done.
62 of 581 done.
63

## 添加词性标注和命名实体识别的特征

还没法用

In [0]:
# from nltk.tag import StanfordNERTagger
# from nltk.tag import StanfordPOSTagger
# from nltk.parse.stanford import StanfordDependencyParser

In [0]:
# import nltk

In [0]:
# nltk.download()

In [0]:
# nerTagger = StanfordNERTagger('./StanfordNLP/nertagger/chinese.misc.distsim.crf.ser.gz', path_to_jar='./StanfordNLP/nertagger/stanford-ner.jar')
# posTagger = StanfordPOSTagger('./StanfordNLP/postagger/chinese-distsim.tagger', path_to_jar="./StanfordNLP/postagger/stanford-postagger.jar")
# dependParser = StanfordDependencyParser(model_path='./StanfordNLP/parser/chinesePCFG.ser.gz', 
#                                         path_to_jar='./StanfordNLP/parser/stanford-parser.jar',
#                                         path_to_models_jar='./StanfordNLP/parser/stanford-parser-3.7.0-models.jar',
#                                         encoding='gbk')

In [0]:
# !pip install pyltp

In [0]:
# from pyltp import Parser

# 训练集XML解析

In [0]:
import numpy as np

In [0]:
from xml.dom.minidom import parse
import xml.dom.minidom

# 使用minidom解析器打开 XML 文档
DOMTree = xml.dom.minidom.parse("rite_test.xml")
collection = DOMTree.documentElement
if collection.hasAttribute("dataset"):
   print ("Root element : %s" % collection.getAttribute("dataset"))

In [0]:
dataset = collection.getElementsByTagName("pair")

In [0]:
pair_set = {'ID':[],'label':[],'revlabel':[],'category':[],'t1':[],'t2':[]}

for pair in dataset:
   ID = pair.getAttribute('id')
   label = pair.getAttribute('label')
   revlabel = pair.getAttribute('revlabel')
   category = pair.getAttribute('category')
   t1 = pair.getElementsByTagName('t1')[0].childNodes[0].data
   t2 = pair.getElementsByTagName('t2')[0].childNodes[0].data
   pair_set['ID'].append(ID)
   pair_set['label'].append(label)
   pair_set['revlabel'].append(revlabel)
   pair_set['category'].append(category)
   pair_set['t1'].append(t1)
   pair_set['t2'].append(t2)

In [0]:
rand_idx = np.random.randint(len(pair_set['ID']))
print ("*****RANDOM PAIR*****")
print ("ID: %s" % pair_set['ID'][rand_idx])
print ("Label: %s" % pair_set['label'][rand_idx])
print ("revlabel: %s" % pair_set['revlabel'][rand_idx])
print ('category: %s' % pair_set['category'][rand_idx])
print ("t1: %s" % pair_set['t1'][rand_idx])
print ("t2: %s" % pair_set['t2'][rand_idx])

*****RANDOM PAIR*****
ID: 413
Label: Y
revlabel: N
category: quantity
t1: 东协自由贸易区包括原东协国家(印尼、马来西亚、菲律宾、新加坡、泰国、汶莱)和4个新成员国(越南、寮国、缅甸、柬埔寨)，共10个国家。
t2: 东协自由贸易区包括原东协国家(印尼、马来西亚、菲律宾、新加坡、泰国、汶莱)和越南、寮国、缅甸、柬埔寨，共10个国家。


In [0]:
sentences_corpus = pair_set['t1'] + pair_set['t2']

# 使用SVM模型

In [0]:
from sklearn import svm

In [46]:
svm_model = svm.SVC()
svm_model.fit(train_data, train_label)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [47]:
svm_result = svm_model.predict(eval_data)
svm_result

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [0]:
# svm_result == np.array(eval_label)
acc = sum(svm_result == np.array(eval_label))/len(eval_label)

In [49]:
acc

0.6612903225806451

In [0]:
def F1_eval(result, eval_label):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  for i in range(len(eval_label)):
    if result[i] == True:
      if eval_label[i] == True:
        TP += 1
      elif eval_label[i] == False:
        FP += 1
    elif result[i] == False:
      if eval_label[i] == True:
        FN += 1
      elif eval_label[i] == False:
        TN += 1
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  F1 = 2 * precision * recall / (precision + recall)
  return F1

In [51]:
svm_F1 = F1_eval(svm_result, eval_label)
svm_F1

0.7899999999999999

# 使用XGBoost模型

In [0]:
import xgboost as xgb
from xgboost import XGBClassifier

In [53]:
xgb_model = XGBClassifier(max_depth = 5, learning_rate = 1e-5, n_estimators = 500, #弱分类树太少的话取不到更多的特征重要性
                   silent=True, objective='binary:logistic')
xgb_model.fit(np.array(train_data), np.array(train_label))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=1e-05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [0]:
xgb_result = xgb_model.predict(np.array(eval_data))
xgb_acc = sum(xgb_result == np.array(eval_label))/len(eval_label)

In [55]:
xgb_result

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [56]:
xgb_acc

0.6532258064516129

In [57]:
xgb_F1 = F1_eval(xgb_result, eval_label)
xgb_F1

0.7772020725388601

# 测试集数据读取

In [0]:
import numpy as np
import jieba

In [0]:
from xml.dom.minidom import parse
import xml.dom.minidom

# 使用minidom解析器打开 XML 文档
test_path = 'rite2014.xml'
DOMTree = xml.dom.minidom.parse(test_path)
collection = DOMTree.documentElement
if collection.hasAttribute("dataset"):
   print ("Root element : %s" % collection.getAttribute("dataset"))

In [0]:
test_dataset = collection.getElementsByTagName("pair")

In [0]:
test_pair_set = {'ID':[],'t1':[],'t2':[]}

for pair in test_dataset: # 测试集没有后面的标签
   ID = pair.getAttribute('id')
   # label = pair.getAttribute('label')
   # revlabel = pair.getAttribute('revlabel')
   # category = pair.getAttribute('category')
   t1 = pair.getElementsByTagName('t1')[0].childNodes[0].data
   t2 = pair.getElementsByTagName('t2')[0].childNodes[0].data
   test_pair_set['ID'].append(ID)
   # test_pair_set['label'].append(label)
   # test_pair_set['revlabel'].append(revlabel)
   # test_pair_set['category'].append(category)
   test_pair_set['t1'].append(t1)
   test_pair_set['t2'].append(t2)

In [61]:
rand_idx = np.random.randint(len(test_pair_set['ID']))
print ("*****RANDOM PAIR*****")
print ("ID: %s" % test_pair_set['ID'][rand_idx])
# print ("Label: %s" % test_pair_set['label'][rand_idx])
# print ("revlabel: %s" % test_pair_set['revlabel'][rand_idx])
# print ('category: %s' % test_pair_set['category'][rand_idx])
print ("t1: %s" % test_pair_set['t1'][rand_idx])
print ("t2: %s" % test_pair_set['t2'][rand_idx])

*****RANDOM PAIR*****
ID: 631
t1: 柯布-道格拉斯生产函数于1900年至1928年间经过Charles Cobb和Paul Douglas的统计验证后确立，并因而得名。
t2: 函数的统计验证可能需要数十年的时间。


In [0]:
sentences_corpus = test_pair_set['t1'] + test_pair_set['t2']

In [63]:
test_data = []
for i in range(len(test_pair_set['ID'])):
  t1 = list(jieba.cut(test_pair_set['t1'][i]))
  t2 = list(jieba.cut(test_pair_set['t2'][i]))
  fea1 = index_bigram_overlap(t1, t2, bigram)
  fea2 = index_cosine_score(t1, t2)
  fea3 = index_longest_overlap_substring(t1, t2)
  fea4 = index_synonym(t1, t2, synonyms)
  fea5 = index_antonym(t1, t2, antonyms)
  fea6 = index_and(t1, t2)
  fea7 = index_or(t1, t2)
  fea8 = index_neg(t1, t2)
  fea9 = index_possible(t1, t2)
  test_data.append([fea1, fea2, fea3, fea4, fea5, fea6, fea7, fea8, fea9])
  print('{0} of {1} done.'.format(i, len(test_pair_set['ID'])))

0 of 1200 done.
1 of 1200 done.
2 of 1200 done.
3 of 1200 done.
4 of 1200 done.
5 of 1200 done.
6 of 1200 done.
7 of 1200 done.
8 of 1200 done.
9 of 1200 done.
10 of 1200 done.
11 of 1200 done.
12 of 1200 done.
13 of 1200 done.
14 of 1200 done.
15 of 1200 done.
16 of 1200 done.
17 of 1200 done.
18 of 1200 done.
19 of 1200 done.
20 of 1200 done.
21 of 1200 done.
22 of 1200 done.
23 of 1200 done.
24 of 1200 done.
25 of 1200 done.
26 of 1200 done.
27 of 1200 done.
28 of 1200 done.
29 of 1200 done.
30 of 1200 done.
31 of 1200 done.
32 of 1200 done.
33 of 1200 done.
34 of 1200 done.
35 of 1200 done.
36 of 1200 done.
37 of 1200 done.
38 of 1200 done.
39 of 1200 done.
40 of 1200 done.
41 of 1200 done.
42 of 1200 done.
43 of 1200 done.
44 of 1200 done.
45 of 1200 done.
46 of 1200 done.
47 of 1200 done.
48 of 1200 done.
49 of 1200 done.
50 of 1200 done.
51 of 1200 done.
52 of 1200 done.
53 of 1200 done.
54 of 1200 done.
55 of 1200 done.
56 of 1200 done.
57 of 1200 done.
58 of 1200 done.
59 of 1

# 测试集上结果

In [0]:
svm_model = 

In [67]:
svm_test_result = svm_model.predict(test_data)
svm_test_result

[ True  True  True ...  True  True  True]


In [75]:
xgb_test_result = xgb_model.predict(test_data)
xgb_test_result

array([ True,  True,  True, ...,  True,  True,  True])

In [0]:
with open('result.txt','w') as result_file:
  for i in range(len(svm_test_result)):
    if(svm_test_result[i]==True):
      ans = "\tY\n"
    else:
      ans = "\tN\n"
    result_text = str(test_pair_set["ID"][i]) + ans
    result_file.write(result_text)

In [0]:
with open('2017211395-2017211438.txt','w') as result_file:
  for i in range(len(xgb_test_result)):
    if(xgb_test_result[i]==True):
      ans = "\tY\n"
    else:
      ans = "\tN\n"
    result_text = str(test_pair_set["ID"][i]) + ans
    result_file.write(result_text)

建议使用xgb的结果作为预测，因为svm把所有例子都预测成正了。

# 参考  
[《基于多特征的中文文本蕴含识别》](http://www.doc88.com/p-5426908131506.html)